In [5]:
# takes two list of urls as input, applies models and generates score of being fraud and clone,
# returns two lists of dictionaries

from ipynb.fs.full.Clone_Logo import clone_logo
from ipynb.fs.full.Clone_Desc import clone_desc
from ipynb.fs.full.Fraud_Desc import fraud_desc
from ipynb.fs.full.Fraud_Vers import fraud_vers
from ipynb.fs.full.Fraud_Reply import fraud_reply
from ipynb.fs.full.Fraud_Dev import fraud_dev
from ipynb.fs.full.Fraud_Metrics import fraud_metrics
from ipynb.fs.full.Fraud_Review import fraud_review

def driver_2(id_inputs, id_database):

    flist = []
    clist = []

    fraud_version_list = fraud_vers(id_inputs)
    fraud_reply_list = fraud_reply(id_inputs)
    i = 0

    for app_id in id_inputs:
        # run app_id with models

        ifdict = dict()
        icdict = dict()   

        dpath1 = 'buffers/logo1.png'
        dpath2 = 'buffers/logo2.png'

        # clone:
        
        cval_logo, cval_logo_id = clone_logo(app_id,id_database,dpath1,dpath2)
        cval_desc, cval_desc_id = clone_desc(app_id,id_database)

        cflag = 1
        if cval_logo == 0 and cval_desc != 0 :
            cval_total = cval_desc
            cval_max = 'Matching Description'
            cval_id = cval_desc_id
        elif cval_desc == 0 and cval_logo != 0:
            cval_total = cval_logo
            cval_max = 'Matching Logo'
            cval_id = cval_logo_id
        elif cval_desc != 0 and cval_logo != 0:
            if cval_logo > cval_desc:
                cval_total = cval_logo
                cval_max = 'Matching Logo'
                cval_id = cval_logo_id
            else:
                cval_total = cval_desc
                cval_max = 'Matching Description'
                cval_id = cval_desc_id
        else:
            cval_id = 0
            cval_max = 0
            cval_total = 0
            cflag = 0

        icdict['App_Id_Clone'] = app_id
        icdict['App_Id_Real'] = cval_id
        icdict['App_Id_Reason'] = cval_max
        icdict['App_Id_Attribute'] = cval_total

        # fraud:

        fval_review = fraud_review(app_id)
        fval_metrics = fraud_metrics(app_id)
        fval_verion = fraud_version_list[i]
        fval_reply = fraud_reply_list[i]
        fval_description = fraud_desc(app_id)
        fval_developer = fraud_dev(app_id,id_database)
        # fval_permission = 0

        fval_total = 0
        fval_max = ''

        fflag = 1
        
        # weightage and fflag

        ifdict['App_Id'] = app_id
        ifdict['Reason'] = fval_max 
        ifdict['Probability'] = fval_total

        if fflag:
            flist.append(ifdict)
        if cflag:
            clist.append(icdict)

        i += 1

    return flist, clist